In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from NeuralNet import PCA
import pandas as pd
import numpy as np

def fetch(f):
    df = pd.read_csv(f, header=None, index_col=0)
    labels = df.pop(1).eq('M').mul(1)
    mean = df.mean()
    std = df.std()
    data = (df - mean) / std
    X, y = data.values.astype(float), labels.values.astype(int)
    y = y.reshape(-1, 1)
    y = np.array([[0, 1] if x else [1, 0] for x in y])
    return X, y, mean, std

In [ ]:
X, y, mean, std = fetch("data_training.csv")

In [ ]:
idx = np.random.permutation(X.shape[0])
X, y = X[idx], y[idx]

In [ ]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(30, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 2)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [ ]:
net = Net()

In [ ]:
print(net)

In [ ]:
test = torch.randn(4, 30)

In [ ]:
net.forward(test)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.1)

In [ ]:
for i in range(1000):
    #zeroing gradients
    optimizer.zero_grad()
    
    #shuffling dataset
    idx = np.random.permutation(X.shape[0])
    X = X[idx]
    y = y[idx]
    
    out = net.forward(X)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {i} -- loss: {loss.item()}")

In [ ]:
def fetch_test(f, mean, std):
    df = pd.read_csv(f, header=None, index_col=0)
    labels = df.pop(1).eq('M').mul(1)
    data = (df - mean) / std
    X, y = data.values.astype(float), labels.values.astype(int)
    y = y.reshape(-1, 1)
    y = np.array([[0, 1] if x else [1, 0] for x in y])
    return X, y

In [ ]:
X_test, y_test = fetch_test("data_test.csv", mean, std)
X_test, y_test = torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()

In [ ]:
out = net.forward(X_test)
loss = criterion(out, y_test)
print("Test loss:", loss.item())

In [ ]:
from NeuralNet import NeuralNet, Dense, softmax

In [ ]:
NN = NeuralNet()

In [ ]:
NN.append(Dense(20, 30))
NN.append(Dense(10, 20))
NN.append(Dense(2, 10, activation=softmax))

In [ ]:
loss, _ = NN.train(X.numpy(), y.numpy(), epoch=1000, rate=0.1, batch=None)

In [ ]:
#calculate loss using pytorch
out = NN.forward(X_test.numpy())
loss = criterion(torch.from_numpy(out).float(), y_test)
print(loss.item())

In [ ]:
#calculate loss using my code
print(NN.loss(X_test.numpy(), y_test.numpy()))